<h3>NLP Tutorial: Text Classification Using FastText</h3>

##### Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

We have a dataset of ecommerce item description. Total 4 categories,
1. Household
2. Electronics
3. Clothing and Accessories
4. Books

The task at hand is to classify a product into one of the above 4 categories based on the product description

In [1]:
import pandas as pd

df= pd.read_csv("data/ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


**Drop NA values**

In [2]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [3]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [4]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When you train a fasttext model, it expects labels to be specified with __label__ prefix. We will just create a third column in the dataframe that has __label__ as well as the product description

In [6]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [7]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


**Pre-procesing**
1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case

In [8]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [9]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [10]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


**Train Test Split**

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [12]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [13]:
train.to_csv("data/ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("data/ecommerce.test", columns=["category_description"], index=False, header=False)

**Train the model and evaluate performance**

In [15]:
import fasttext

model = fasttext.train_supervised(input="data/ecommerce.train")
model.test("data/ecommerce.test")

Read 4M words
Number of words:  78885
Number of labels: 4
Progress: 100.0% words/sec/thread:  749146 lr: -0.000022 avg.loss:  0.197367 ETA:   0h 0m 0s lr:  0.000000 avg.loss:  0.197367 ETA:   0h 0m 0s


(10085, 0.9677739216658403, 0.9677739216658403)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

**Now let's do prediction for few product descriptions**

In [16]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99840045]))

In [17]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [18]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00001001]))

In [19]:
model.get_nearest_neighbors("painting")

[(0.99898761510849, 'fulfilment'),
 (0.9989861845970154, "europa's"),
 (0.9989824295043945, 'jeweler'),
 (0.998982310295105, 'beaders'),
 (0.9989816546440125, 'elementseasy'),
 (0.9989816546440125, 'supportfeatures'),
 (0.9989816546440125, 'providedassembly'),
 (0.9989816546440125, "decor's"),
 (0.9989816546440125, 'durabilityfeatures'),
 (0.9989816546440125, 'needscomes')]

In [20]:
model.get_nearest_neighbors("sony")

[(0.9992310404777527, '2160p30'),
 (0.9992310404777527, 'blackmagic'),
 (0.9992310404777527, 'decklink'),
 (0.9992270469665527, 'piezotm'),
 (0.9992270469665527, 'picturemate'),
 (0.9992167949676514, 'immedaitely'),
 (0.9992167949676514, 'kinking'),
 (0.9992128610610962, 'ad600'),
 (0.9992122054100037, 'arbo'),
 (0.9992098808288574, 'chokes')]

In [21]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, '2168'),
 (0.0, 'hj'),
 (0.0, 'flavourings')]